# AF推定

- 心拍数推定を元に、AF推定を試す

In [1]:
# 使ってないやつもありそうなので、最終的には整理する
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib
import scipy.signal as signal
import scipy.stats as stats
import soundfile as sf
import joblib

from pathlib import Path
from Utils import resample_waveform, spike_removal, hpf, lpf, get_sound_vol, wavelet_denoising, rms_agc_2stages, compress_peak_power,sliding_window_peak_detection, peak_interval_estimation, peak_count_estimation, detect_peaks_with_FFT, peak_variation_estimation


## データ読み込み

In [2]:
# データの読み込み
file_path = os.path.join('..','250624_af_dataset_all.xlsx')
wav_df = pd.read_excel(file_path)

# 抽出とインデックスリセット（必要に応じて指定）
# wav_df = wav_df[wav_df['exp_study'] == 1].reset_index(drop=True)
# wav_df = wav_df[wav_df['is_third_heart_sound'] != 1].reset_index(drop=True)
# wav_df = wav_df[wav_df['AF'] == 0].reset_index(drop=True)

# 列番号258～796を持つデータのみを抽出
# wav_df = wav_df[wav_df['number'].between(258, 796)].reset_index(drop=True)

# AFデータのWAVファイルが格納されているルートディレクトリのパス
wav_root = os.path.join('..', '..', 'wav_af')

# データ数を表示
print(f"データ数: {len(wav_df)}")


データ数: 2020


# process

In [3]:
### parameters ###
window_size_sec = 2 # １はNG
step_size_sec = 1
min_lag_bpm_normal = 120  # ピーク検出の最小bpm(normal)
min_lag_bpm_tachy = 220  # ピーク検出の最小bpm(tachy)
min_lag_coeff_normal = 1/(min_lag_bpm_normal/60)  # ピーク検出の最小時間(normal)
min_lag_coeff_tachy = 1/(min_lag_bpm_tachy/60)  # ピーク検出の最小時間(tachy)

fs = 1000 # 元のサンプリングレートを使う場合はsrを指定(通常1000Hz)
fc_hpf = 16.5  # HPFのカットオフ周波数
fc_lpf = 62  # LPFのカットオフ周波数
k_thr = 0.45  # ウェーブレット分解のしきい値
ma_window_size = 55  # 移動平均のウィンドウサイズ
comp_thr = 0.581  # コンプレッションのしきい値
comp_rat = 0.657  # コンプレッションの比率
prom_coeff_normal = 2.5  # ピーク検出のprominence係数(normal)
prom_coeff_tachy = 0.05  # ピーク検出のprominence係数(tachy)
height_qxx = 50  # ピーク検出のheight係数
height_coeff_normal = 1.5  # ピーク検出のheight係数(normal)
height_coeff_tachy = 0.5  # ピーク検出のheight係数(tachy)

### プロット設定 ###
plot_enable = False  # True:プロット表示, False:プロット非表示
FFT_plot_enable = False

### tachy判定モデルの読み込み ###
rf_model = joblib.load('randomforest_model_selected5_250630.pkl')
rf_imputer = joblib.load('randomforest_imputer_selected5_250630.pkl')

# 結果を格納するデータフレームを作成
all_results = []

""" for loop """
test_run = False  # テスト実行フラグ
# テスト実行時は指定範囲、本番実行時は全件処理
for i in range(0, 2) if test_run else range(len(wav_df)):
    file_name = wav_df['wav_name'][i]  # ファイル名を取得
    file_path = os.path.join(wav_root, file_name)  # wav_rootフォルダ内のファイルパスを指定
    data, sr = sf.read(file_path, dtype='int16')  # soundfileでwavファイルを読み込み

    # Filter
    data_res = resample_waveform(data, sr, fs)  # リサンプリング
    data_spike = spike_removal(data_res, window_size=5)  # スパイク除去
    data_hpf = hpf(data_spike, fs, fc_hpf)  # HPF
    data_lpf = lpf(data_hpf, fs, fc_lpf)  # LPF
    data = data_lpf
    time = np.arange(0, len(data))/fs

    # Sound Volume check
    ma_window = int(0.05 * fs +1)  # 移動平均するサンプル数（固定）
    vol_mean = get_sound_vol(data, fs, ma_window)
    wav_df.loc[i, 'sound_vol'] = vol_mean
    
    # 音量が小さい場合は以降の処理をスキップ
    if vol_mean <= 3000:
        print(f"Skipping {file_name} due to low volume (vol_mean={vol_mean:.1f})")
        continue

    ### Wavelet Denoising ###
    denoised_signal, coeffs_info = wavelet_denoising(data, wavelet='bior6.8', k_thr=k_thr, max_level=7)

    ### RMS-AGC 2-stages ###
    rms_env2_interp, rms_env2_ma = rms_agc_2stages(denoised_signal, fs,
                                  fade_sec=0.5,
                                  hop_size_sec=0.01,
                                  frame_size_sec=0.05,
                                  ma_window_size=ma_window_size,
                                  agc1_params=None,
                                  agc2_params=None)

    ### コンプレッション ###
    rms_env2_ma_comp = rms_env2_ma/(max(rms_env2_ma)+1e-8)
    rms_env2_ma_comp = compress_peak_power(rms_env2_ma_comp, threshold=comp_thr, ratio=comp_rat)

    ### 窓ごとピーク検出 ###
    peak_results = sliding_window_peak_detection(
        signal=rms_env2_ma_comp,
        fs=fs,
        window_size_sec=window_size_sec,
        step_size_sec=step_size_sec,
        min_lag_bpm_normal=min_lag_bpm_normal,
        min_lag_bpm_tachy=min_lag_bpm_tachy,
        prom_coeff_normal=prom_coeff_normal,
        prom_coeff_tachy=prom_coeff_tachy,
        height_qxx=height_qxx,
        height_coeff_normal=height_coeff_normal,
        height_coeff_tachy=height_coeff_tachy,
        plot=plot_enable
        )

    # 結果を取得
    avg_peak_count_normal = peak_results['avg_peak_count_normal']  # 平均ピーク間隔(normal)
    time_peaks_normal = peak_results['time_peaks_normal']  # ピークの時刻(normal)
    avg_peak_count_tachy = peak_results['avg_peak_count_tachy']  # 平均ピーク間隔(tachy)
    time_peaks_tachy = peak_results['time_peaks_tachy']  # ピークの時刻(tachy)


    ### ピークインターバル法で心拍数を推定 ###
    hr_interval_normal, hr_interval_tachy = peak_interval_estimation(
        avg_peak_count_normal,
        avg_peak_count_tachy,
        )

    ### ピークカウント法で心拍数を推定 ###
    hr_count_normal, hr_count_tachy = peak_count_estimation(
        time_peaks_normal,
        time_peaks_tachy,
        rms_env2_ma_comp,
        fs
        )

    ### FFTピーク検出と特徴量抽出（可視化も含む） ###
    save_path = os.path.join('figures', file_name.replace('.wav', '.jpeg'))
    features = detect_peaks_with_FFT(
        signal=rms_env2_interp,
        fs=fs,
        desired_sec=15,
        high_freq_thr=5.5,
        power_thr=0.3,
        prominence_coeff=0.6,
        height_threshold=0.01,
        plot=FFT_plot_enable,
        file_name=file_name,
        note_x=wav_df['note_x'][i],
        vol_mean=vol_mean,
        actual_hr=wav_df['heart_rate'][i],  # 実装時は削除
        # save_path=save_path # 指定するとグラフ保存
    )
    
    ### 特徴量を個別変数として取得 ###
    bpm_first_peak = features['bpm_first_peak']
    bpm_dominant_peak = features['bpm_dominant_peak']
    max_power_under_220 = features['max_power_under_220']
    high_freq_peaks = features['high_freq_peaks']
    bpm_spectrum_centroid = features['bpm_spectrum_centroid']
    
    # FFT特徴量辞書を作成（頻脈判定用）
    FFT_feature_dict = {
        'bpm_first_peak': bpm_first_peak,
        'bpm_dominant_peak': bpm_dominant_peak,
        'max_power_under_220': max_power_under_220,
        'high_freq_peaks': high_freq_peaks,
        'bpm_spectrum_centroid': bpm_spectrum_centroid,    }
    
    ### RandomForestによる頻脈判定 ###
    tachy_features_df = pd.DataFrame(FFT_feature_dict, index=[0])  # 特徴量をDataFrameに変換
    tachy_features_df_imputed = pd.DataFrame(rf_imputer.transform(tachy_features_df), columns=tachy_features_df.columns)  # 列名を保持したまま欠損値を補完
    tachy_flag = rf_model.predict(tachy_features_df_imputed).astype(int)  # 頻脈判定
    tachy_prob = rf_model.predict_proba(tachy_features_df_imputed)[:, 1]  # 頻脈判定確率

    ### FFT法の心拍数推定結果 ###
    # 心拍数上限を200とする。
    # normal
    hr_FFT_normal = bpm_first_peak
    if hr_FFT_normal > 200: 
        hr_FFT_normal = int(hr_FFT_normal/2)  # 200以上は2で割る
    else:
        hr_FFT_normal = int(hr_FFT_normal)
    
    # tachy
    hr_FFT_tachy = bpm_dominant_peak
    if hr_FFT_tachy > 200:  
        hr_FFT_tachy = int(hr_FFT_tachy/2)  # 200以上は2で割る
    else:
        hr_FFT_tachy = int(hr_FFT_tachy)

    ### 統合手法で心拍数を推定 ###
    hr_list_normal = [hr_interval_normal, hr_count_normal, hr_FFT_normal]  
    hr_list_tachy = [hr_interval_tachy, hr_count_tachy, hr_FFT_tachy]
    
    if tachy_flag == 0:
        hr_integrated = int(np.median(hr_list_normal))
    elif tachy_flag == 1:
        hr_integrated = hr_FFT_tachy
    else:
        hr_integrated = np.nan

    ### 各手法の結果を辞書として保存 ###
    hr = wav_df['heart_rate'][i]

    ### ピーク間隔のばらつきを計算 ###
    # 確認のため、計算に使用するピーク時刻と間隔を出力
    variation, time_peaks_normal_unique, intervals_normal, \
    time_peaks_tachy_unique, intervals_tachy = \
    peak_variation_estimation(
        time_peaks_normal,
        time_peaks_tachy
    )

    ### 結果を辞書としてまとめる ###
    results_data = {
        'number': i,  # 通し番号
        'wav_name': file_name,
        'note_x': wav_df['note_x'][i],
        'is_third_heart_sound': wav_df['is_third_heart_sound'][i],
        'sound_vol': vol_mean,
        'heart_rate': hr,
        'hr_interval_normal': hr_interval_normal,
        'hr_interval_tachy': hr_interval_tachy,
        'hr_count_normal': hr_count_normal,
        'hr_count_tachy': hr_count_tachy,
        'FFT_first': bpm_first_peak,
        'FFT_dominant': bpm_dominant_peak,
        'hr_FFT_normal': hr_FFT_normal,
        'hr_FFT_tachy': hr_FFT_tachy,
        'hr_integrated': hr_integrated,
        'tachy_flag': tachy_flag,
        'tachy_prob': tachy_prob,
        'AF': wav_df['AF'][i],
        'peak_std_normal': variation['normal']['std'],
        'peak_std_tachy': variation['tachy']['std'],
        'peak_mad_normal': variation['normal']['mad'],
        'peak_mad_tachy': variation['tachy']['mad'],
        'peak_var_normal': variation['normal']['var'],
        'peak_var_tachy': variation['tachy']['var'],
        'peak_rmssd_normal': variation['normal']['rmssd'],
        'peak_rmssd_tachy': variation['tachy']['rmssd'],
        'peak_nn50_normal': variation['normal']['nn50'],
        'peak_nn50_tachy': variation['tachy']['nn50'],
        'peak_nn20_normal': variation['normal']['nn20'],
        'peak_nn20_tachy': variation['tachy']['nn20'],
        'peak_pnn50_normal': variation['normal']['pnn50'],
        'peak_pnn50_tachy': variation['tachy']['pnn50'],
        'peak_pnn20_normal': variation['normal']['pnn20'],
        'peak_pnn20_tachy': variation['tachy']['pnn20'],
        }
    all_results.append(results_data)

    print(f"ファイル：{file_name} 処理完了, 推定：{hr_integrated}, 頻脈判定：{tachy_flag}, NN分散：{variation['normal']['var']:0.3f}, {variation['tachy']['var']:0.3f}")
    print(f"note_x：{wav_df['note_x'][i]}")

all_results_df = pd.DataFrame(all_results)

### データフレームを保存 ###
# resultフォルダが無い場合は作成
if not os.path.exists('result'):
    os.makedirs('result')
all_results_df.to_csv('result/af_test_250704.csv', encoding='utf-8-sig', index=False)
all_results_df.to_pickle('result/af_test_250704.pkl')
print("結果を保存しました。")


ファイル：3002_20240114070258_163.wav 処理完了, 推定：84, 頻脈判定：[0], NN分散：0.032, 0.040
note_x：正常
ファイル：1003_20240126081339_365.wav 処理完了, 推定：96, 頻脈判定：[0], NN分散：0.001, 0.020
note_x：正常
ファイル：1003_20240126180226_375.wav 処理完了, 推定：94, 頻脈判定：[0], NN分散：0.010, 0.004
note_x：正常
ファイル：1004_20240202132845_632.wav 処理完了, 推定：74, 頻脈判定：[0], NN分散：0.065, 0.022
note_x：正常
ファイル：3005_20240202143120_634.wav 処理完了, 推定：76, 頻脈判定：[0], NN分散：0.024, 0.008
note_x：正常
ファイル：1004_20240206190416_832.wav 処理完了, 推定：78, 頻脈判定：[0], NN分散：0.003, 0.026
note_x：正常
ファイル：1004_20240208083659_892.wav 処理完了, 推定：73, 頻脈判定：[0], NN分散：0.086, 0.012
note_x：正常
ファイル：4002_20240209174433_964.wav 処理完了, 推定：60, 頻脈判定：[0], NN分散：0.032, 0.086
note_x：正常
Skipping 4002_20240210184148_1024.wav due to low volume (vol_mean=2984.0)
ファイル：1005_20240212073036_1090.wav 処理完了, 推定：68, 頻脈判定：[0], NN分散：0.070, 0.049
note_x：とてもきれいに撮れている。低調な収縮期雑音あり。S3なし。
ファイル：3009_20240228134341_1967.wav 処理完了, 推定：60, 頻脈判定：[0], NN分散：0.089, 0.043
note_x：正常
ファイル：3006_20240229070350_1990.wav 処理完了, 推定：60, 頻脈判定：[0], 